<div class="alert-warning">
 <font size="+2"> <b> Raw data alignment </b> </font>
</div>

This tutorial will cover the steps for performing the alignment of raw RNA- and HiFi-sequencing data. You will need to use [the software IGV](https://software.broadinstitute.org/software/igv/download) on your computer to visualize some of the output files, which can be easily downloaded once they are produced. At the end of this tutorial you will be able to:
- perform and discuss quality control on raw data in `fastq` format using `FastQC` and `MultiQC`
- align HiFi and RNA sequencing data with dedicated tools such as `MiniMap2` and `STAR`
- analyze the quality the alignment with `qualimap` 
The output of this notebook will be used for the Variant calling analysis and the bulk RNA-sequencing analysis. If you do not want to run this notebook, you can alternatively use [the free interactive tool Galaxy](https://usegalaxy.org/) to perform the alignment steps. We have uploaded the data on Galaxy, and the manual to perform the exercise is found at the course webpage !!!!!!!!!!!!!!!.

The present tutorial, like the rest of the course material, is available at our [open-source github repository](https://github.com/hds-sandbox/NGS_summer_course_Aarhus.


A few introductory points to run this notebook:

* To use this notebook, use the `NGS (python)` kernel that contains the packages. Choose it by selecting `Kernel -> Change Kernel` in the menu on top of the window.

<figure>
<img src="images/kernelchoice.png" width="700" alt="Kernel Choice" class="center">
</figure>

* In this notebook you will use only bash commands as you would do in the command line (this is why you read `%%bash` at the beginning of each piece of code). Those commands can be replicated in the command line, but we thought of integrating them in a notebook to make the tutorial understandable. The bash commands can also be marked with an `!` sign at the beginning of the line
* On some computers, you might see the result of the commands once they are done running. This means you will wait some time while the computer is crunching, and only afterwards you will see the result of the command you have executed
*  You can run the code in each cell by clicking on the run cell button, or by pressing <kbd> Shift </kbd> + <kbd> Enter </kbd>.  When the code is done running, a small green check sign will appear on the left side
*   You need to run the cells in sequential order, please do not run a cell until the one above finished running and do not skip any cells
*   Each cell contains a short description of the code and the output you should get. Please try not to focus on understanding the code for each command in too much detail, but rather try to focus on the output 
*   You can create new code cells by pressing <kbd> + </kbd> in the Menu bar above. 


# Biological background

White clover (Trifolium repens) is an allotetraploid.  It is a relatively young, outcrossing species, which originated during the most recent glaciation around 20,000 years ago by hybridisation of two diploid species, T. occidentale and T. pallescens (see figure below). 

<figure>
<img src="images/white_clover.png" width="700" alt="Kernel Choice" class="center">
</figure>

This means that it contains genomes originating from two different species within the same nucleus. Normally, white clover is an outbreeding species, but a self-compatible line was used for sequencing the white clover genome [(Griffiths et al, 2019)](https://academic.oup.com/plcell/article/31/7/1466/5985684). This line will be 
designated as `S10` in the data, indicating that this is the 10th self-fertilized generation. In addition, we have data from a wild clover accession (ecotype) called Tienshan (`Ti`), which is collected from the Chinese mountains and is adapted to alpine conditions.

We will perform alignment of the data to the white clover's reference genome containing both T. occidentale and T. pallescens (called `contig 1` and `contig 2` in the data). We will also perform alignment to each subgenome, and see which are the differences with the quality control tools.

Check if the data folder is there

In [1]:
%%bash
sh ../Scripts/dataDownloadScript.sh

Copying datasets from https://zenodo.org/record/6952995


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  145M  100  145M    0     0   216k      0  0:11:27  0:11:27 --:--:--  355k


Clover_Data/
Clover_Data/S10_1_1.R1.fastq
Clover_Data/Hifi_reads_white_clover.fastq
Clover_Data/DNA_Contig1.fasta
Clover_Data/TI_1_2.R1.fastq
Clover_Data/TI_1_1.R2.fastq
Clover_Data/DNA_Contig1_2.fasta.fai
Clover_Data/S10_2_2.R1.fastq
Clover_Data/S10_2_3.R2.fastq
Clover_Data/Functional_Annotations.txt
Clover_Data/white_clover_genes.gff
Clover_Data/TI_2_1.R2.fastq
Clover_Data/TI_1_3.R2.fastq
Clover_Data/S10_2_1.R1.fastq
Clover_Data/DNA_Contig2.fasta
Clover_Data/DNA_Contig2.fasta.fai
Clover_Data/TI_2_3.R2.fastq
Clover_Data/S10_1_2.R2.fastq
Clover_Data/S10_1_3.R2.fastq
Clover_Data/TI_2_2.R2.fastq
Clover_Data/TI_1_1.R1.fastq
Clover_Data/TI_2_2.R1.fastq
Clover_Data/metadata.csv
Clover_Data/TI_2_3.R1.fastq
Clover_Data/TI_2_1.R1.fastq
Clover_Data/TI_1_2.R2.fastq
Clover_Data/S10_2_2.R2.fastq
Clover_Data/S10_2_1.R2.fastq
Clover_Data/DNA_Contig1.fasta.fai
Clover_Data/S10_1_3.R1.fastq
Clover_Data/S10_2_3.R1.fastq
Clover_Data/S10_1_1.R2.fastq
Clover_Data/TI_1_3.R1.fastq
Clover_Data/DNA_Contig1_2.f

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


scrna_data/
scrna_data/crypto_azoospermia.h5ad
scrna_data/rawDataScanpy.h5ad


Task exception was never retrieved
future: <Task finished name='Task-8' coro=<ScriptMagics.shebang.<locals>._handle_stream() done, defined at /opt/conda/envs/NGS_aarhus/lib/python3.10/site-packages/IPython/core/magics/script.py:211> exception=ValueError('Separator is not found, and chunk exceed the limit')>
Traceback (most recent call last):
  File "/opt/conda/envs/NGS_aarhus/lib/python3.10/asyncio/streams.py", line 525, in readline
    line = await self.readuntil(sep)
  File "/opt/conda/envs/NGS_aarhus/lib/python3.10/asyncio/streams.py", line 603, in readuntil
    raise exceptions.LimitOverrunError(
asyncio.exceptions.LimitOverrunError: Separator is not found, and chunk exceed the limit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/NGS_aarhus/lib/python3.10/site-packages/IPython/core/magics/script.py", line 213, in _handle_stream
    line = (await stream.readline()).decode("utf8")
  File "/opt/conda/env

# Quality control and mapping

## Quality Control

We run `FastQC` on the PacBio Hifi reads and on two of the Illumina RNA-seq libraries. `FastQC` does quality control of the raw sequence data, providing an overview of the data which can help identify if there are any problems that should be addressed before further analysis. You can find the report for each file into the folder `results/fastqc_output/`. The output is in HTML format and can be opened in any browser or in `jupyterlab`. It is however not easy to compare the various libraries by opening separate reports. To aggregate all the results, we apply the `MultiQC` software to the reports' folder. The output of MultiQC is in the directory `results/multiqc_output/fastqc_data`.

In [2]:
%%bash
#run fast
mkdir -p results/fastqc_output
fastqc -q -o results/fastqc_output ../Data/Clover_Data/*.fastq

In [3]:
%%bash
multiqc --outdir results/multiqc_output/fastqc_data results/fastqc_output


  /// MultiQC 🔍 | v1.13.dev0

|           multiqc | Search path : /usr/Material/Notebooks/results/fastqc_output
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 50/50  
|            fastqc | Found 25 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : results/multiqc_output/fastqc_data/multiqc_report.html
|           multiqc | Data        : results/multiqc_output/fastqc_data/multiqc_data
|           multiqc | MultiQC complete


<div class="alert-success"> <font size="+2"> <b> Questions </b> </font> </div>

Visualize the Webpage generated by MultiQC.

Hint: You can find a `Help` button that offers additional information about the plots for each panel. Focus on the following panels: “Per base sequence quality”, “Per sequence quality scores”.... (“Per base sequence content” always gives a `FAIL` for RNA-seq data).

* What do you notice with respect to the sequence quality scores? 
* Are there any other quality issues worth noting?

## Hifi data mapping

We map the PacBio Hifi reads (`Hifi_reads_white_clover.fastq`) to the white clover reference sequence (Contig1&2) using `minimap2`. We run two mapping rounds, using two different preset options (`-x` in the command) for the technology:
* PacBio/Oxford Nanopore read to reference mapping: `map-pb`
* Long assembly to reference mapping. Divergence is far below 5%” settings `asm5`.
Next, we create reports of the mapping results by running `QualiMap` on the two obtained SAM files.

We first need to index the reference fasta files using `samtools faidx`. This produces files in `.fai` format containing informations about length of the reference sequence, offset for the quality scores, name of the reference sequence. [Click here](http://www.htslib.org/doc/faidx.html) for a detailed overview. 

In [4]:
%%bash
samtools faidx ../Data/Clover_Data/DNA_Contig1_2.fasta
samtools faidx ../Data/Clover_Data/DNA_Contig1.fasta
samtools faidx ../Data/Clover_Data/DNA_Contig2.fasta

we create an output folder for the HIFI alignment, and run `minimap2` with the settings explained before.

In [5]:
%%bash 
mkdir -p results/HIFI_alignment/
minimap2 -a -x map-pb -o results/HIFI_alignment/PacBio_clover_alignment_1_2_mappb.sam \
                            ../Data/Clover_Data/DNA_Contig1_2.fasta \
                            ../Data/Clover_Data/Hifi_reads_white_clover.fastq 

minimap2 -a -x asm5 -o results/HIFI_alignment/PacBio_clover_alignment_1_2_asm5.sam \
                            ../Data/Clover_Data/DNA_Contig1_2.fasta \
                            ../Data/Clover_Data/Hifi_reads_white_clover.fastq

[M::mm_idx_gen::0.071*1.04] collected minimizers
[M::mm_idx_gen::0.278*0.42] sorted minimizers
[M::main::0.278*0.42] loaded/built the index for 2 target sequence(s)
[M::mm_mapopt_update::0.288*0.44] mid_occ = 11
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 2
[M::mm_idx_stat::0.295*0.45] distinct minimizers: 203943 (79.05% are singletons); average occurrences: 1.273; average spacing: 8.047; total length: 2089554
[M::worker_pipeline::33.583*2.72] mapped 4395 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -a -x map-pb -o results/HIFI_alignment/PacBio_clover_alignment_1_2_mappb.sam ../Data/Clover_Data/DNA_Contig1_2.fasta ../Data/Clover_Data/Hifi_reads_white_clover.fastq
[M::main] Real time: 33.680 sec; CPU: 91.409 sec; Peak RSS: 1.152 GB
[M::mm_idx_gen::0.062*1.08] collected minimizers
[M::mm_idx_gen::0.089*1.37] sorted minimizers
[M::main::0.089*1.37] loaded/built the index for 2 target sequence(s)
[M::mm_mapopt_update::0.096*1.34] mid_occ = 50
[M::mm_idx_st

`samtools sort` is used to sort the alignment with left-to-right coordinates. The output is in `.bam` format, with `.sam` files in input (Note that you could have gotten `.bam` files from `minimap2` with a specific option).

In [6]:
%%bash
samtools sort results/HIFI_alignment/PacBio_clover_alignment_1_2_mappb.sam \
                -o results/HIFI_alignment/PacBio_clover_alignment_1_2_mappb.sort.bam

samtools sort results/HIFI_alignment/PacBio_clover_alignment_1_2_asm5.sam \
                -o results/HIFI_alignment/PacBio_clover_alignment_1_2_asm5.sort.bam

`samtools index` creates the index for the `bam` file, stored in `.bai` format. The index file lets programs access any position into the aligned data without reading the whole file, which would take too much time.

In [7]:
%%bash
samtools index results/HIFI_alignment/PacBio_clover_alignment_1_2_mappb.sort.bam
samtools index results/HIFI_alignment/PacBio_clover_alignment_1_2_asm5.sort.bam

Run quality control on both files

In [8]:
%%bash
qualimap bamqc -bam results/HIFI_alignment/PacBio_clover_alignment_1_2_mappb.sort.bam \
                 -outdir results/qualimap_output/PacBio_clover_alignment_1_2_mappb

qualimap bamqc -bam results/HIFI_alignment/PacBio_clover_alignment_1_2_asm5.sort.bam \
                 -outdir results/qualimap_output/PacBio_clover_alignment_1_2_asm5

Java memory size is set to 1200M
Launching application...

QualiMap v.2.2.2-dev
Built on 2019-11-11 14:05

Selected tool: bamqc
Available memory (Mb): 33
Max memory (Mb): 1258
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 401
Chunk of reads size: 1000
Number of threads: 4
Processed 50 out of 401 windows...
Processed 100 out of 401 windows...
Processed 150 out of 401 windows...
Processed 200 out of 401 windows...
Processed 250 out of 401 windows...
Processed 300 out of 401 windows...
Processed 350 out of 401 windows...
Processed 400 out of 401 windows...
Total processed windows:401
Number of reads: 4395
Number of valid reads: 4696
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 4395
Num mapped first of pair: 0
Num mapped second of pair: 0
Num singletons: 0
Time taken to analyze reads: 7
Computing descriptors...
numberOfMappedBases: 70274383
referenceSize: 2089554
numberOfSeq

<div class="alert-success"> <font size="+2"> <b> Questions </b> </font> </div>

Visualize the reports generated by QualiMap. If you want, you can also inspect the alignment files in `IGV`. First, you will need to download the reference fasta sequence in `../Data/Clover_Data/DNA_Contig1_2.fasta` and import it into IGV. Go to IGV and use the `Genomes --> Load Genome from file` menu and select the relevant fasta file. Afterwards, to view the alignments directly in IGV click on the Visualize button on the selected dataset (marked in red) and choose the `1. display with IGV (local)` option. You need to have IGV opened locally on your computer. 

* What do you observe when comparing the two BAM files? 
* Which setting do you think is the more appropriate to use, and why?
* Have a look at the polymorphic regions in IGV and think about if they are true polymorphisms.

Next, we map the white clover PacBio Hifi reads to contig1 and contig2 separately, using the setting you selected at the previous step (let's say `map-pb` was chosen, but you are free to change this setting in the commands). As the two contigs represent the two white clover subgenomes, this mapping will allow you to see the two subgenome haplotypes and call subgenome SNPs.


In [9]:
%%bash 
minimap2 -a -x map-pb -o results/HIFI_alignment/PacBio_clover_alignment_1.sam \
                            ../Data/Clover_Data/DNA_Contig1.fasta \
                            ../Data/Clover_Data/Hifi_reads_white_clover.fastq

[M::mm_idx_gen::0.052*1.12] collected minimizers
[M::mm_idx_gen::0.068*1.37] sorted minimizers
[M::main::0.068*1.37] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.072*1.35] mid_occ = 10
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.077*1.32] distinct minimizers: 112547 (91.45% are singletons); average occurrences: 1.141; average spacing: 8.031; total length: 1031631
[M::worker_pipeline::49.182*2.83] mapped 4395 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -a -x map-pb -o results/HIFI_alignment/PacBio_clover_alignment_1.sam ../Data/Clover_Data/DNA_Contig1.fasta ../Data/Clover_Data/Hifi_reads_white_clover.fastq
[M::main] Real time: 49.205 sec; CPU: 139.289 sec; Peak RSS: 2.212 GB


In [10]:
%%bash 
minimap2 -a -x map-pb -o results/HIFI_alignment/PacBio_clover_alignment_2.sam \
                            ../Data/Clover_Data/DNA_Contig2.fasta \
                            ../Data/Clover_Data/Hifi_reads_white_clover.fastq

[M::mm_idx_gen::0.075*1.06] collected minimizers
[M::mm_idx_gen::0.114*1.38] sorted minimizers
[M::main::0.114*1.38] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.117*1.37] mid_occ = 10
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.122*1.35] distinct minimizers: 120722 (93.18% are singletons); average occurrences: 1.087; average spacing: 8.062; total length: 1057923
[M::worker_pipeline::40.863*2.92] mapped 4395 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -a -x map-pb -o results/HIFI_alignment/PacBio_clover_alignment_2.sam ../Data/Clover_Data/DNA_Contig2.fasta ../Data/Clover_Data/Hifi_reads_white_clover.fastq
[M::main] Real time: 40.912 sec; CPU: 119.274 sec; Peak RSS: 2.016 GB


Sort the bam files and create their index using `samtools`

In [11]:
%%bash
samtools sort results/HIFI_alignment/PacBio_clover_alignment_1.sam -o results/HIFI_alignment/PacBio_clover_alignment_1.sort.bam
samtools sort results/HIFI_alignment/PacBio_clover_alignment_2.sam -o results/HIFI_alignment/PacBio_clover_alignment_2.sort.bam

In [12]:
%%bash
samtools index results/HIFI_alignment/PacBio_clover_alignment_1.sort.bam
samtools index results/HIFI_alignment/PacBio_clover_alignment_2.sort.bam

Perform quality control

In [13]:
%%bash
mkdir -p results/qualimap_output
qualimap bamqc -bam results/HIFI_alignment/PacBio_clover_alignment_1.sort.bam -outdir results/qualimap_output/PacBio_clover_alignment_1

Java memory size is set to 1200M
Launching application...

QualiMap v.2.2.2-dev
Built on 2019-11-11 14:05

Selected tool: bamqc
Available memory (Mb): 33
Max memory (Mb): 1258
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 400
Chunk of reads size: 1000
Number of threads: 4
Processed 50 out of 400 windows...
Processed 100 out of 400 windows...
Processed 150 out of 400 windows...
Processed 200 out of 400 windows...
Processed 250 out of 400 windows...
Processed 300 out of 400 windows...
Processed 350 out of 400 windows...
Processed 400 out of 400 windows...
Total processed windows:400
Number of reads: 4395
Number of valid reads: 9070
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 4356
Num mapped first of pair: 0
Num mapped second of pair: 0
Num singletons: 0
Time taken to analyze reads: 9
Computing descriptors...
numberOfMappedBases: 59058058
referenceSize: 1031631
numberOfSeq

In [14]:
%%bash
qualimap bamqc -bam results/HIFI_alignment/PacBio_clover_alignment_2.sort.bam -outdir results/qualimap_output/PacBio_clover_alignment_2

Java memory size is set to 1200M
Launching application...

QualiMap v.2.2.2-dev
Built on 2019-11-11 14:05

Selected tool: bamqc
Available memory (Mb): 33
Max memory (Mb): 1258
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 400
Chunk of reads size: 1000
Number of threads: 4
Processed 50 out of 400 windows...
Processed 100 out of 400 windows...
Processed 150 out of 400 windows...
Processed 200 out of 400 windows...
Processed 250 out of 400 windows...
Processed 300 out of 400 windows...
Processed 350 out of 400 windows...
Processed 400 out of 400 windows...
Total processed windows:400
Number of reads: 4395
Number of valid reads: 9462
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 4394
Num mapped first of pair: 0
Num mapped second of pair: 0
Num singletons: 0
Time taken to analyze reads: 9
Computing descriptors...
numberOfMappedBases: 59388453
referenceSize: 1057923
numberOfSeq

<div class="alert-success"> <font size="+2"> <b> Questions </b> </font> </div>

Visualize all three alignments (HiFi reads mapped to Contigs1_2, contig1 and contig2) in IGV. If you want to load BAM files in IGV directly from your computer, you will also need to index the
files.

* Why do you see fluctuations in coverage and large regions without any apparent subgenome SNPs?
* What are the major differences between the stats for the reads mapped to Contigs1&2 versus contig1 and contig2? What is your interpretation of the differences?

# RNA-seq mapping

In the `../Data` folder you will find 24 RNA-seq libraries, 12 `S10` libraries and 12  `Tienshan` libraries. Each library is paired-end, which is denoted by `R1` and `R2` at the end of two files having the same name, such as `S10_1_1.R1.fastq` and `S10_1_1.R2.fastq`. We will align each library separately and then merge the alignments to create two final samples for `S10` and `Tienshan`.

First, we need to create a genome file for the reference fasta file of contigs `1` and `2`. This is done with `STAR`, using the option `--runMode genomeGenerate`. Look at the code for reading what other option are for. `STAR` is a very complex tool with many options, so it is always useful to [have a reference manual](https://physiology.med.cornell.edu/faculty/skrabanek/lab/angsd/lecture_notes/STARmanual.pdf).

In [15]:
%%bash
STAR --runThreadN 8 \
--runMode genomeGenerate \
--genomeDir results/STAR_output/indexing_contigs_1_2 \
--genomeFastaFiles ../Data/Clover_Data/DNA_Contig1_2.fasta \
--sjdbGTFfile ../Data/Clover_Data/white_clover_genes.gtf \

	STAR --runThreadN 8 --runMode genomeGenerate --genomeDir results/STAR_output/indexing_contigs_1_2 --genomeFastaFiles ../Data/Clover_Data/DNA_Contig1_2.fasta --sjdbGTFfile ../Data/Clover_Data/white_clover_genes.gff --quantMode GeneCounts
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Aug 15 11:56:29 ..... started STAR run
Aug 15 11:56:29 ... starting to generate Genome files
Aug 15 11:56:29 ..... processing annotations GTF


!!!!! WARNING: --genomeSAindexNbases 14 is too large for the genome size=2089554, which may cause seg-fault at the mapping step. Re-run genome generation with recommended --genomeSAindexNbases 9


Aug 15 11:56:29 ... starting to sort Suffix Array. This may take a long time...
Aug 15 11:56:29 ... sorting Suffix Array chunks and saving them to disk...
Aug 15 11:56:30 ... loading chunks from disk, packing SA...
Aug 15 11:56:30 ... finished generating suffix array
Aug 15 11:56:30 ... generating Suffix Array index
Aug 15 11:56:34 ... completed Suffix Array index
Aug 15 11:56:34 ... writing Genome to disk ...
Aug 15 11:56:34 ... writing Suffix Array to disk ...
Aug 15 11:56:34 ... writing SAindex to disk
Aug 15 11:56:49 ..... finished successfully


We got a warning saying
```
!!!!! WARNING: --genomeSAindexNbases 14 is too large for the genome size=2089554, which may cause seg-fault at the mapping step. Re-run genome generation with recommended --genomeSAindexNbases 9
```
so we rerun the command with the suggested option.

In [18]:
!rm -rf results/STAR_output/indexing_contigs_1_2

In [1]:
!gffread ../Data/Clover_Data/white_clover_genes.gff -T -o ../Data/Clover_Data/white_clover_genes.gtf

In [19]:
%%bash
STAR --runThreadN 8 \
--runMode genomeGenerate \
--genomeDir results/STAR_output/indexing_contigs_1_2 \
--genomeFastaFiles ../Data/Clover_Data/DNA_Contig1_2.fasta \
--sjdbGTFfile ../Data/Clover_Data/white_clover_genes.gtf \
--genomeSAindexNbases 9

	STAR --runThreadN 8 --runMode genomeGenerate --genomeDir results/STAR_output/indexing_contigs_1_2 --genomeFastaFiles ../Data/Clover_Data/DNA_Contig1_2.fasta --sjdbGTFfile ../Data/Clover_Data/white_clover_genes.gtf --genomeSAindexNbases 9
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Aug 15 15:06:32 ..... started STAR run
Aug 15 15:06:32 ... starting to generate Genome files
Aug 15 15:06:32 ..... processing annotations GTF
Aug 15 15:06:32 ... starting to sort Suffix Array. This may take a long time...
Aug 15 15:06:32 ... sorting Suffix Array chunks and saving them to disk...
Aug 15 15:06:33 ... loading chunks from disk, packing SA...
Aug 15 15:06:33 ... finished generating suffix array
Aug 15 15:06:33 ... generating Suffix Array index
Aug 15 15:06:33 ... completed Suffix Array index
Aug 15 15:06:33 ..... inserting junctions into the genome indices
Aug 15 15:06:33 ... writing Genome to disk ...
Aug 15 15:06:33 ... writing

We use again `STAR` to align every single library for `S10`. We extract the library name of each file and run STAR through each pair of files. Note that plant introns are very rarely more than `5000 bp` and that you are mapping to two homoeologous contigs that show high similarity, especially in genic regions. We set the maximum size to 5000 using `--alignIntronMax 5000`.

In [37]:
!ls -lh ../Data/Clover_Data/S10*.R1.fastq

-rw-r--r-- 1 ucloud users 18M Aug  2 14:05 ../Data/Clover_Data/S10_1_1.R1.fastq
-rw-r--r-- 1 ucloud users 25M Aug  2 14:05 ../Data/Clover_Data/S10_1_2.R1.fastq
-rw-r--r-- 1 ucloud users 26M Aug  2 14:05 ../Data/Clover_Data/S10_1_3.R1.fastq
-rw-r--r-- 1 ucloud users 26M Aug  2 14:05 ../Data/Clover_Data/S10_2_1.R1.fastq
-rw-r--r-- 1 ucloud users 24M Aug  2 14:05 ../Data/Clover_Data/S10_2_2.R1.fastq
-rw-r--r-- 1 ucloud users 27M Aug  2 14:06 ../Data/Clover_Data/S10_2_3.R1.fastq


In [20]:
!rm -rf results/STAR_output/S10_align_contigs_1_2

In [21]:
%%bash
for i in `ls ../Data/Clover_Data/S10*.R1.fastq`
do

PREFIXNAME=`basename $i .R1.fastq`
echo "###############################################"
echo "##### ALIGNING PAIRED-END READS "$PREFIXNAME
echo "###############################################"
STAR --genomeDir results/STAR_output/indexing_contigs_1_2/ \
--runThreadN 8 \
--runMode alignReads \
--readFilesIn ../Data/Clover_Data/$PREFIXNAME.R1.fastq ../Data/Clover_Data/$PREFIXNAME.R2.fastq \
--outFileNamePrefix results/STAR_output/S10_align_contigs_1_2/$PREFIXNAME \
--outSAMtype BAM SortedByCoordinate \
--outSAMattributes Standard \
--quantMode GeneCounts \
--alignIntronMax 5000 \
--genomeSAindexNbases 9 

done

###############################################
##### ALIGNING PAIRED-END READS S10_1_1
###############################################
	STAR --genomeDir results/STAR_output/indexing_contigs_1_2/ --runThreadN 8 --runMode alignReads --readFilesIn ../Data/Clover_Data/S10_1_1.R1.fastq ../Data/Clover_Data/S10_1_1.R2.fastq --outFileNamePrefix results/STAR_output/S10_align_contigs_1_2/S10_1_1 --outSAMtype BAM SortedByCoordinate --outSAMattributes Standard --quantMode GeneCounts --alignIntronMax 5000 --genomeSAindexNbases 9
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Aug 15 15:06:42 ..... started STAR run
Aug 15 15:06:42 ..... loading genome
Aug 15 15:06:42 ..... started mapping
Aug 15 15:06:52 ..... finished mapping
Aug 15 15:06:52 ..... started sorting BAM
Aug 15 15:06:52 ..... finished successfully
###############################################
##### ALIGNING PAIRED-END READS S10_1_2
######################################

Do the same alignment for `Tienshan` libraries

In [16]:
!rm -rf results/STAR_output/TI_align_contigs_1_2

In [17]:
%%bash
for i in `ls ../Data/Clover_Data/TI*.R1.fastq`
do

PREFIXNAME=`basename $i .R1.fastq`
echo "###############################################"
echo "##### ALIGNING PAIRED-END READS "$PREFIXNAME
echo "###############################################"
STAR --genomeDir results/STAR_output/indexing_contigs_1_2/ \
--runThreadN 8 \
--readFilesIn ../Data/Clover_Data/$PREFIXNAME.R1.fastq ../Data/Clover_Data/$PREFIXNAME.R2.fastq \
--outFileNamePrefix results/STAR_output/TI_align_contigs_1_2/$PREFIXNAME \
--outSAMtype BAM SortedByCoordinate \
--outSAMattributes Standard \
--quantMode GeneCounts \
--alignIntronMax 5000 \
--genomeSAindexNbases 9 

done

###############################################
##### ALIGNING PAIRED-END READS TI_1_1
###############################################
	STAR --genomeDir results/STAR_output/indexing_contigs_1_2/ --runThreadN 8 --readFilesIn ../Data/Clover_Data/TI_1_1.R1.fastq ../Data/Clover_Data/TI_1_1.R2.fastq --outFileNamePrefix results/STAR_output/TI_align_contigs_1_2/TI_1_1 --outSAMtype BAM SortedByCoordinate --outSAMattributes Standard --quantMode GeneCounts --alignIntronMax 5000 --genomeSAindexNbases 9
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Aug 15 14:46:42 ..... started STAR run
Aug 15 14:46:42 ..... loading genome
Aug 15 14:46:42 ..... started mapping
Aug 15 14:46:53 ..... finished mapping
Aug 15 14:46:53 ..... started sorting BAM
Aug 15 14:46:53 ..... finished successfully
###############################################
##### ALIGNING PAIRED-END READS TI_1_2
###############################################
	STAR --genomeDir

Run quality control on each aligned library with `MultiQC`. In this way there will be a whole report to compare `S10` files and `Tienshan` files.

In [20]:
%%bash
multiqc --outdir results/multiqc_output/TI_STAR_align_1_2 \
            results/STAR_output/TI_align_contigs_1_2/


  /// MultiQC 🔍 | v1.13.dev0

|           multiqc | Search path : /usr/Material/Notebooks/results/STAR_output/TI_align_contigs_1_2
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 78/78  
|              star | Found 6 reports and 6 gene count files
|           multiqc | Compressing plot data
|           multiqc | Report      : results/multiqc_output/TI_STAR_align_1_2/multiqc_report.html
|           multiqc | Data        : results/multiqc_output/TI_STAR_align_1_2/multiqc_data
|           multiqc | MultiQC complete


In [21]:
%%bash
multiqc --outdir results/multiqc_output/S10_STAR_align_1_2 \
            results/STAR_output/S10_align_contigs_1_2/


  /// MultiQC 🔍 | v1.13.dev0

|           multiqc | Search path : /usr/Material/Notebooks/results/STAR_output/S10_align_contigs_1_2
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 78/78  
|              star | Found 6 reports and 6 gene count files
|           multiqc | Compressing plot data
|           multiqc | Report      : results/multiqc_output/S10_STAR_align_1_2/multiqc_report.html
|           multiqc | Data        : results/multiqc_output/S10_STAR_align_1_2/multiqc_data
|           multiqc | MultiQC complete


We merge the outputs of each group of aligned libraries. Here is how the files look like for the `Tienshan`.

In [22]:
!ls -lh  results/STAR_output/TI_align_contigs_1_2/TI_*.sortedByCoord.out.bam 

-rw-r--r-- 1 ucloud users 6.6M Aug 15 11:57 results/STAR_output/TI_align_contigs_1_2/TI_1_1Aligned.sortedByCoord.out.bam
-rw-r--r-- 1 ucloud users 6.0M Aug 15 11:57 results/STAR_output/TI_align_contigs_1_2/TI_1_2Aligned.sortedByCoord.out.bam
-rw-r--r-- 1 ucloud users 7.1M Aug 15 11:57 results/STAR_output/TI_align_contigs_1_2/TI_1_3Aligned.sortedByCoord.out.bam
-rw-r--r-- 1 ucloud users 9.6M Aug 15 11:57 results/STAR_output/TI_align_contigs_1_2/TI_2_1Aligned.sortedByCoord.out.bam
-rw-r--r-- 1 ucloud users 7.2M Aug 15 11:58 results/STAR_output/TI_align_contigs_1_2/TI_2_2Aligned.sortedByCoord.out.bam
-rw-r--r-- 1 ucloud users 8.2M Aug 15 11:58 results/STAR_output/TI_align_contigs_1_2/TI_2_3Aligned.sortedByCoord.out.bam


Apply `samtools merge`

In [26]:
%%bash
mkdir -p results/STAR_output/TI_align_contigs_1_2_merge/
samtools merge -fo results/STAR_output/TI_align_contigs_1_2_merge/TI.sorted.bam results/STAR_output/TI_align_contigs_1_2/TI_*.sortedByCoord.out.bam 

In [24]:
%%bash
mkdir -p results/STAR_output/S10_align_contigs_1_2_merge/
samtools merge -fo results/STAR_output/S10_align_contigs_1_2_merge/S10.sorted.bam results/STAR_output/S10_align_contigs_1_2/S10_*.sortedByCoord.out.bam 

Index both merging outputs

In [25]:
%%bash
samtools index results/STAR_output/S10_align_contigs_1_2_merge/S10.sorted.bam results/STAR_output/TI_align_contigs_1_2_merge/TI.sorted.bam

# Wrapping up  🎉 🎉 🎉

In this exercise, you learnt to align various types of data after performing quality control for raw data. We looked at some of the options for the aligners and at how to use some of the basic samtools manipulation programs. The outputs from the RNA alignments will be used for the VCF file analysis in the next notebook, and the RNA alignments will be use for the bulk RNA data analysis.

